<a href="https://colab.research.google.com/github/AhrendsW/Pipeline-de-Dados-na-AWS-e-Gerenciamento-de-Custos/blob/main/Pipeline_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p data

In [ ]:
import urllib.request

def extract_data(url, dest):
    try:
        urllib.request.urlretrieve(url, dest)
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
extract_data('https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/tmphrybkxuh.csv', 'data/data_2015.csv')
extract_data('https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/tmpzxzxeqfb.csv', 'data/data_2016.csv')
extract_data('https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/tmpzccn8u4q.csv', 'data/data_2017.csv')
extract_data('https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv', 'data/data_2018.csv')
extract_data('https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv', 'data/data_2019.csv')
extract_data('https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv', 'data/data_2020.csv')

In [ ]:
import glob
arquivos_csv = glob.glob('data/*.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install python-dotenv -qq

In [ ]:
from dotenv import load_dotenv
import os
dotenv_path = '/content/drive/MyDrive/Curso-AWS/Pipeline-Intro/.config/.env'
load_dotenv(dotenv_path)

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_REGION')
bucket_name = os.getenv('BUCKET_NAME')

In [ ]:
!pip install boto3 -qq

Conectando a AWS pelo SDK boto3

In [ ]:
import boto3
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

In [ ]:
# Para apagar os arquivos caso necessário

def remover_arquivos_s3(bucket_name, prefix):
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if 'Contents' in response:
        for obj in response['Contents']:
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])

remover_arquivos_s3(bucket_name, 'bronze/')

Salvar arquivos em Parquet

In [ ]:
from io import BytesIO
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

for arquivo in arquivos_csv:
    ano = os.path.basename(arquivo).split('_')[1].split('.')[0]
    df = pd.read_csv(arquivo)
    parquet_buffer = BytesIO()

    table = pa.Table.from_pandas(df)
    pq.write_table(table, parquet_buffer)

    parquet_buffer.seek(0)
    s3_client.upload_fileobj(parquet_buffer, bucket_name, f'bronze/data_{ano}.parquet')


In [ ]:
# Verificar conteúdo
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='bronze/')
keys = [obj['Key'] for obj in response.get('Contents', [])]
keys

['bronze/data_2015.parquet',
 'bronze/data_2016.parquet',
 'bronze/data_2017.parquet',
 'bronze/data_2018.parquet',
 'bronze/data_2019.parquet',
 'bronze/data_2020.parquet']